### RNN Cell




模块名称  : 作用




$x_t$:
    表时刻$t$时的输入数据


RNN Cell:
本质上是一个线性层


$h_t$:
表时刻$t$时得到的输出（隐含层）

RNN Cell为一个线性层Linear，在$t$时刻下的$N$维向量，经过Cell后即可变为一个$M$维的向量$h_t$,而与其他线性层不同，RNN Cell为一个共享的线性层。即重复利用，权重共享。

由于$x_1 \cdots x_n$，为一组序列信息，每一个$x_i$都至少应包含$x_{i-1}$的信息。也就是说，针对$x_2$的操作所得到的$h_2$中，应当包含$x_1$的信息，因此在设计中，把$x_1$处理后得到的$h_1$一并向下传递。
$h_0$是一种前置信息。例如若实现图像到文本的转换，可以利用CNN+FC(全连接层)对图像进行操作，再将输出的向量作为$h_0$参与到RNN的运算中。
若没有可获知的前置信息，可将$h_0$设置为与$x_i$同维度的零向量。
RNN Cell为同一个Linear，即让设计好的Linear反复参与运算，实现权重共享。

---
## RNN Cell 计算过程

符号 :
    标注




$h_t$ $h_{t-1}$
:    隐藏层hidden的结果向量


$x_t$
:    输入层的输入向量


$R ^{hidden\_size}$
:表隐藏层的向量维度


$R^{input\_size}$
:表输入层的向量维度


$W_{ih}$
:用于计算输入的权重，维度大小为$hidden\_size \times input\_size$


$b_{ih}$
:用于计算输入时的偏置量


$W_{hh}$
:用于计算隐藏层的权重，维度大小为$hidden\_size \times hidden\_size$


$b_{hh}$
:用于计算隐藏层时的偏置量


tanh
:激活函数，值域为$ (-1, +1)$


+
:求和模块

---
在RNN计算过程中，分别对输入$x_t$以及前文的隐藏层输出$h_{t-1}$进行线性计算，再进行求和，对所得到的一维向量，利用tanh激活函数进行激活，由此可以得到当前隐藏层的输出$h_t$,其计算过程如下：

$h_t = tanh(W_{ih}x_t + b_{ih} + W_{hh}h_{t-1} + b_{hh})$

实际上RNN Cell的计算过程中为线性计算。

$W_{hh}h_{t-1}+W_{ih}x_{t} = 
\begin{bmatrix}
{W_{hh}}&{W_{ih}}
\end{bmatrix}
\begin{bmatrix}
h\\
x
\end{bmatrix}$

即在实际运算的过程中，这两部分是拼接到一起形成矩阵再计算求和的，最终形成一个大小为$hidden\_size \times 1$的张量。

---
## 代码实现

代码实现有两种模式，一是实现自己的RNN Cell，再自己重写循环调用等逻辑。二是直接调用RNN的网络。

### 重点在于控制其输入输出的维度。

## RNN Cell

In [ ]:
import torch

batch_size = 1          # batchsize
seq_len = 3             # 序列长度为3（样本个数） ex：x1,x2,x3
input_size = 4          # 输入维度               ex: x1,x2,x3都是四维向量
hidden_size =2          # 隐藏层为2              ex：每一个hidden都是一个2维向量

#输入维度input_size,隐藏层维度hidden_size
cell = torch.nn.RNNCell(input_size = input_size, hidden_size = hidden_size)    # 构造RNNCell，分别表示两个输入的维度（就可以确定权重的维度和偏置的维度） 
#维度最重要
dataset = torch.randn(seq_len,batch_size,input_size)   # 整个序列维度（序列的长度 批量 输入x的维度）（序列的长度第一个是将来循环时，每次要先拿出当前时刻t的一组张量（BATCH_SIZE, INPUT_SIZE)）
#初始化时设为零向量
hidden = torch.zeros(batch_size, hidden_size)          # 隐藏层初始全0  

for idx,input in enumerate(dataset):
    print('=' * 20,idx,'=' * 20)
    print('Input size:', input.shape)
    #输入的input 的维度（B*input_size）, hidden的维度（B*hidden_size）
    #输出的hidden维度（B*hidden_szie）
    hidden = cell(input, hidden)      # 当前时刻的输入和当前的hidden

    print('outputs size: ', hidden.shape)
    print(hidden)

## RNN

In [ ]:
import torch

batch_size = 1          # batchsize
seq_len = 5             # 序列长度为3（样本个数） ex：x1,x2,x3
input_size = 4          # 输入维度               ex: x1,x2,x3都是四维向量
hidden_size =2          # 隐藏层为2              ex：每一个hidden都是一个2维向量
num_layers = 3          # RNN有多少层

#其他参数
#batch_first=True 维度从(SeqLen*Batch*input_size)变为（Batch*SeqLen*input_size）
#说明input维度，hidden维度，以及RNN层数
#RNN计算耗时大，不建议层数过深
cell = torch.nn.RNN(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers)       # 最后是RNN有多少层
#inputs指的是X1……Xn的整个输入序列
#hidden指的是前置条件H0
#out指的是每一次迭代的H1……Hn隐藏层序列
#hidden(跟out一起的)指的是最后一次迭代得到输出Hn
inputs = torch.randn(seq_len, batch_size, input_size)          # （序列长度 批量 输入维度）
hidden = torch.zeros(num_layers, batch_size, hidden_size)      # （numLayers 批量 隐藏层维度）

out, hidden = cell(inputs, hidden)    # 右边：dataset表示整个输入序列，hidden就是开始h0
                                      # 左边：out是h1-hn,hidden就是hn
print("Output size: ", out.shape)
print("Output: ", out)
print("Hidden size: ", hidden.shape)
print("Hidden: ", hidden)

---
# 参考实例

现在有一个序列到序列$（seq \to seq）$的任务，比如将“hello”转换为“ohlol”。

## RNN Cell

In [13]:
import torch
import numpy as np

input_size = 4
hidden_size = 4
batch_size = 1
num_layers = 1
seq_len = 5

#构建输入输出字典
idx2char = ['e', 'h', 'l', 'o'] # 字典
x_data = [1, 0, 2, 2, 3] # hello
y_data = [3, 1, 2, 3, 2] # ohlol
# y_data = [3, 1, 2, 2, 3]

one_hot_lookup = [[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]]

#构造独热向量，此时向量维度为(SeqLen*InputSize)
x_one_hot = [one_hot_lookup[x] for x in x_data]
#view(-1……)保留原始SeqLen，并添加batch_size,input_size两个维度
inputs = torch.Tensor(x_one_hot).view(-1, batch_size, input_size)
#将labels转换为（SeqLen*1）的维度
labels = torch.LongTensor(y_data).view(-1, 1)

class Model(torch.nn.Module):
    def __init__(self, input_size, hidden_size, batch_size):
        super(Model, self).__init__()
        self.batch_size = batch_size
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.rnncell = torch.nn.RNNCell(input_size = self.input_size,
                                        hidden_size = self.hidden_size)

    def forward(self, input, hidden):
        # RNNCell input = (batchsize*inputsize)
        # RNNCell hidden = (batchsize*hiddensize)
        hidden = self.rnncell(input, hidden)
        return hidden

    #初始化零向量作为h0，只有此处用到batch_size
    def init_hidden(self):
        return torch.zeros(self.batch_size, self.hidden_size)    

net = Model(input_size, hidden_size, batch_size)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)

for epoch in range(15):
    #损失及梯度置0，创建前置条件h0
    loss = 0
    optimizer.zero_grad()
    hidden = net.init_hidden()

    print("Predicted string: ",end="")
    #inputs=（seqLen*batchsize*input_size） labels = (seqLen*1)
    #input是按序列取的inputs元素（batchsize*inputsize）
    #label是按序列去的labels元素（1）
    for input, label in zip(inputs, labels):
        hidden = net(input, hidden)
        #序列的每一项损失都需要累加
        loss = + criterion(hidden, label)
        #多分类取最大
        _, idx = hidden.max(dim=1)
        print(idx2char[idx.item()], end='')

    loss.backward()
    optimizer.step()

    print(", Epoch [%d/15] loss = %.4f" % (epoch+1, loss.item()))

Predicted string: loooo, Epoch [1/15] loss = 1.2609
Predicted string: lolll, Epoch [2/15] loss = 0.8008
Predicted string: lllll, Epoch [3/15] loss = 0.5528
Predicted string: lllll, Epoch [4/15] loss = 0.4354
Predicted string: lllll, Epoch [5/15] loss = 0.3854
Predicted string: lllll, Epoch [6/15] loss = 0.3640
Predicted string: lllll, Epoch [7/15] loss = 0.3540
Predicted string: lllll, Epoch [8/15] loss = 0.3489
Predicted string: lllll, Epoch [9/15] loss = 0.3461
Predicted string: lllll, Epoch [10/15] loss = 0.3444
Predicted string: lllll, Epoch [11/15] loss = 0.3433
Predicted string: lllll, Epoch [12/15] loss = 0.3426
Predicted string: lllll, Epoch [13/15] loss = 0.3422
Predicted string: lllll, Epoch [14/15] loss = 0.3418
Predicted string: lllll, Epoch [15/15] loss = 0.3416


## RNN

In [14]:
import torch

input_size = 4
hidden_size = 4
batch_size = 1
num_layers = 1
seq_len = 5

#构建输入输出字典
idx2char = ['e', 'h', 'l', 'o'] # 字典
x_data = [1, 0, 2, 2, 3] # hello
y_data = [3, 1, 2, 3, 2] # ohlol
# y_data = [3, 1, 2, 2, 3]

one_hot_lookup = [[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]]

x_one_hot = [one_hot_lookup[x] for x in x_data]

inputs = torch.Tensor(x_one_hot).view(seq_len, batch_size, input_size)
#labels（seqLen*batchSize,1）为了之后进行矩阵运算，计算交叉熵
labels = torch.LongTensor(y_data)

class Model(torch.nn.Module):
    def __init__(self, input_size, hidden_size, batch_size, num_layers=1):
        super(Model, self).__init__()
        self.batch_size = batch_size #构造H0
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = torch.nn.RNN(input_size = self.input_size,
                                hidden_size = self.hidden_size,
                                num_layers=num_layers)

    def forward(self, input):
        hidden = torch.zeros(self.num_layers,
                             self.batch_size,
                             self.hidden_size)
        out, _ = self.rnn(input, hidden)
        #reshape成（SeqLen*batchsize,hiddensize）便于在进行交叉熵计算时可以以矩阵进行。
        return out.view(-1, self.hidden_size)

net = Model(input_size, hidden_size, batch_size, num_layers)


criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)

#RNN中的输入（SeqLen*batchsize*inputsize）
#RNN中的输出（SeqLen*batchsize*hiddensize）
#labels维度 hiddensize*1
for epoch in range(15):
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    _, idx = outputs.max(dim=1)
    idx = idx.data.numpy()
    print('Predicted string: ',''.join([idx2char[x] for x in idx]), end = '')
    print(", Epoch [%d/15] loss = %.3f" % (epoch+1, loss.item()))

Predicted string:  eelll, Epoch [1/15] loss = 1.484
Predicted string:  ellll, Epoch [2/15] loss = 1.318
Predicted string:  ellll, Epoch [3/15] loss = 1.194
Predicted string:  ellll, Epoch [4/15] loss = 1.102
Predicted string:  ollll, Epoch [5/15] loss = 1.029
Predicted string:  ollol, Epoch [6/15] loss = 0.965
Predicted string:  ohlol, Epoch [7/15] loss = 0.907
Predicted string:  ohlol, Epoch [8/15] loss = 0.856
Predicted string:  ohlol, Epoch [9/15] loss = 0.810
Predicted string:  ohlol, Epoch [10/15] loss = 0.767
Predicted string:  ohlol, Epoch [11/15] loss = 0.728
Predicted string:  ohlol, Epoch [12/15] loss = 0.694
Predicted string:  ohlol, Epoch [13/15] loss = 0.665
Predicted string:  ohlol, Epoch [14/15] loss = 0.642
Predicted string:  ohlol, Epoch [15/15] loss = 0.624



独热编码在实际问题中容易引起很多问题：

    独热编码向量维度过高，每增加一个不同的数据，就要增加一维   
    独热编码向量稀疏，每个向量是一个为1其余为0   
    独热编码是硬编码，编码情况与数据特征无关   

## Embedding

目的是为了对数据进行降维,增加Embedding层实现降维，增加线性层使之在处理输入输出维度不同的情况下更加稳定。其中的Embedding层的输入必须是LongTensor类型。

In [15]:
import torch

input_size = 4
num_class = 4
hidden_size = 8
embedding_size =10     # 数据嵌入层10维空间
batch_size = 1
num_layers = 2
seq_len = 5

idx2char_1 = ['e', 'h', 'l', 'o']
# idx2char_2 = ['h', 'l', 'o']

x_data = [[1, 0, 2, 2, 3]]#(batch,seq_len)
y_data = [3, 1, 2, 3, 2] # ohlol#(batch_size*seq_len)

#inputs 作为交叉熵中的Inputs，维度为（batchsize，seqLen）
inputs = torch.LongTensor(x_data)
#labels 作为交叉熵中的Target，维度为（batchsize*seqLen）
labels = torch.LongTensor(y_data)

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self .emb = torch.nn.Embedding(input_size, embedding_size) # （输入大小*embedding_size）

        self.rnn = torch.nn.RNN(input_size = embedding_size,
                                hidden_size = hidden_size,
                                num_layers=num_layers,
                                batch_first = True
                               )
                                
        self.fc = torch.nn.Linear(hidden_size, num_class)
    def forward(self, x):
        hidden = torch.zeros(num_layers, x.size(0), hidden_size)
        x = self.emb(x)# batch_size,seq_len,embedding_size
        x, _ = self.rnn(x, hidden)
        x = self.fc(x)
        return x.view(-1, num_class)

net = Model()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)

for epoch in range(15):
    optimizer.zero_grad()
    outputs = net(inputs)

    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    _, idx = outputs.max(dim=1)
    idx = idx.data.numpy()
    print('Predicted string: ',''.join([idx2char_1[x] for x in idx]), end = '')
    print(", Epoch [%d/15] loss = %.3f" % (epoch+1, loss.item()))

Predicted string:  eeeee, Epoch [1/15] loss = 1.476
Predicted string:  ohlol, Epoch [2/15] loss = 1.121
Predicted string:  ohlll, Epoch [3/15] loss = 0.913
Predicted string:  ohlol, Epoch [4/15] loss = 0.726
Predicted string:  ohlol, Epoch [5/15] loss = 0.554
Predicted string:  ohlol, Epoch [6/15] loss = 0.418
Predicted string:  ohlol, Epoch [7/15] loss = 0.322
Predicted string:  ohlol, Epoch [8/15] loss = 0.255
Predicted string:  ohlol, Epoch [9/15] loss = 0.201
Predicted string:  ohlol, Epoch [10/15] loss = 0.155
Predicted string:  ohlol, Epoch [11/15] loss = 0.116
Predicted string:  ohlol, Epoch [12/15] loss = 0.086
Predicted string:  ohlol, Epoch [13/15] loss = 0.067
Predicted string:  ohlol, Epoch [14/15] loss = 0.054
Predicted string:  ohlol, Epoch [15/15] loss = 0.041
